### Imports

In [25]:
import sys
import os
sys.path.append(os.path.abspath("../.."))
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
from tinyshift.drift import CatDrift, ConDrift
import numpy as np
from tinyshift.stats import chebyshev_guaranteed_percentage
from tinyshift.series import hurst_exponent, foreca
from numpy.random import standard_normal

### Data

In [26]:
n_samples = 100000 
n_features = 20  
n_informative = 2 
n_redundant = 2  
weights = [0.2, 0.8]  
np.random.seed(42)
X, y = make_classification(n_samples=n_samples, 
                           n_features=n_features, 
                           n_informative=n_informative, 
                           n_redundant=n_redundant, 
                           flip_y=0.05, 
                           random_state=42,
                           )

category_col = np.random.choice([0, 1], size=n_samples, p=weights)

diverse_category_col = np.random.choice([0, 1, 2, 3, 4], size=n_samples)


In [27]:
def generate_random_dates_within_interval(start_date, years, n_samples):
    """
    Generates a set of random dates within a specified interval.

    Parameters:
    - start_date: The starting date (datetime object).
    - years: The number of years for the interval (int).
    - n_samples: The number of random date samples to generate (int).

    Returns:
    - np.array: Array of randomly sampled dates within the interval.
    """
    np.random.seed(42)

    weeks = years * 52
    result = np.empty(n_samples, dtype="datetime64[s]")
    current_date = start_date

    for i in range(0, n_samples, n_samples // weeks):
        date_range = pd.date_range(start=current_date, periods=7, freq="D")
        segment_size = min(n_samples // weeks, n_samples - i)
        result[i : i + segment_size] = np.random.choice(
            date_range, size=segment_size, replace=True
        )
        current_date += timedelta(7)

    return result

In [28]:
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(n_features)])
df["unique_id"] = 1
df['discrete_0'] = category_col
df['discrete_1'] = diverse_category_col
df['target'] = y
df['datetime'] = generate_random_dates_within_interval(datetime(2021, 1, 1), 4, n_samples)
df_train = df[df["datetime"] < '2024']
df_reference = df[(df["datetime"] >= '2024-01-01') & (df["datetime"] < '2024-07-01')].copy()
df_test = df[(df["datetime"] >= '2024-07-01')].copy()

In [ ]:
np.random.seed(42)
n_replace = int(len(df) * 0.05)
idx = df.sample(n=n_replace, random_state=42).index
df.loc[idx, "unique_id"] = 2
df_train = df[df["datetime"] < "2024"]
df_reference = df[(df["datetime"] >= "2024-01-01") & (df["datetime"] < "2024-07-01")].copy()
df_test = df[(df["datetime"] >= "2024-07-01")].copy()

Atribuídos unique_id=2 a 5000 linhas (5.00%)
unique_id
1    95000
2     5000
Name: count, dtype: int64


In [30]:
X_train = df_train[df_train.columns[:-2]]
y_train = df_train["target"]

### Training

In [31]:
rf = RandomForestClassifier(random_state=42, oob_score=True, n_jobs=-1, class_weight="balanced")
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_jobs=-1, oob_score=True,
                       random_state=42)

### df_test

In [32]:
y_prob = rf.predict_proba(df_test[df_test.columns[:-2]])
y_pred = rf.predict(df_test[df_test.columns[:-2]])
y_prob = y_prob[np.arange(len(df_test)), y_pred]
df_test["y_prob"] = y_prob
df_test["prediction"] = y_pred

### df_reference

In [33]:
y_prob = rf.predict_proba(df_reference[df_reference.columns[:-2]])
y_pred = rf.predict(df_reference[df_reference.columns[:-2]])
y_prob = y_prob[np.arange(len(y_prob)), y_pred]
df_reference["y_prob"] = y_prob
df_reference["prediction"] = y_pred

# Discrete Data Drift

### Median Absolute Deviation (MAD)

In [46]:
%%time
tracker = CatDrift(df_reference, func="chebyshev", freq="W", drift_limit='mad', time_col="datetime", target_col="discrete_1")

CPU times: user 9.45 ms, sys: 0 ns, total: 9.45 ms
Wall time: 8.36 ms


In [47]:
%%time
tracker.predict(df_reference, time_col="datetime", id_col="unique_id", target_col="discrete_1")

CPU times: user 5.34 ms, sys: 3.39 ms, total: 8.73 ms
Wall time: 7.72 ms


unique_id   datetime    metric  drift
0           1 2024-01-07  0.023623  False
1           1 2024-01-14  0.032022  False
2           1 2024-01-21  0.027684  False
3           1 2024-01-28  0.043360  False
4           1 2024-02-04  0.032039  False
5           1 2024-02-11  0.026642  False
6           1 2024-02-18  0.042232  False
7           1 2024-02-25  0.031039  False
8           1 2024-03-03  0.018809  False
9           1 2024-03-10  0.036373  False
10          1 2024-03-17  0.039445  False
11          1 2024-03-24  0.054376  False
12          1 2024-03-31  0.034119  False
13          1 2024-04-07  0.022519  False
14          1 2024-04-14  0.032718  False
15          1 2024-04-21  0.044505  False
16          1 2024-04-28  0.069950   True
17          1 2024-05-05  0.014421  False
18          1 2024-05-12  0.028390  False
19          1 2024-05-19  0.045724  False
20          1 2024-05-26  0.022381  False
21          1 2024-06-02  0.037573  False
22          1 2024-06-09  0.036546  False
23          1 2024-06-16  0.025065  False
24          1 2024-06-23  0.053635  False
25          1 2024-06-30  0.026890  False
26          2 2024-01-07  0.297592   True
27          2 2024-01-14  0.370852   True
28          2 2024-01-21  0.379389   True
29          2 2024-01-28  0.259131   True
30          2 2024-02-04  0.251370   True
31          2 2024-02-11  0.311129   True
32          2 2024-02-18  0.251270   True
33          2 2024-02-25  0.402165   True
34          2 2024-03-03  0.240400   True
35          2 2024-03-10  0.263227   True
36          2 2024-03-17  0.302400   True
37          2 2024-03-24  0.311829   True
38          2 2024-03-31  0.276214   True
39          2 2024-04-07  0.390500   True
40          2 2024-04-14  0.250570   True
41          2 2024-04-21  0.275614   True
42          2 2024-04-28  0.378589   True
43          2 2024-05-05  0.305689   True
44          2 2024-05-12  0.248565   True
45          2 2024-05-19  0.266262   True
46          2 2024-05-26  0.229900   True
47          2 2024-06-02  0.282067   True
48          2 2024-06-09  0.289700   True
49          2 2024-06-16  0.252858   True
50          2 2024-06-23  0.286196   True
51          2 2024-06-30  0.338326   True

#### Reference Plot

In [36]:
tracker.predict(df_test, time_col="datetime", id_col="unique_id", target_col="discrete_1")

unique_id   datetime    metric  drift
0           1 2024-07-07  0.024383  False
1           1 2024-07-14  0.026516  False
2           1 2024-07-21  0.034076  False
3           1 2024-07-28  0.029449  False
4           1 2024-08-04  0.039414  False
5           1 2024-08-11  0.034282  False
6           1 2024-08-18  0.027887  False
7           1 2024-08-25  0.040502  False
8           1 2024-09-01  0.032947  False
9           1 2024-09-08  0.036093  False
10          1 2024-09-15  0.040735  False
11          1 2024-09-22  0.038698  False
12          1 2024-09-29  0.036883  False
13          1 2024-10-06  0.026520  False
14          1 2024-10-13  0.032404  False
15          1 2024-10-20  0.043508  False
16          1 2024-10-27  0.023933  False
17          1 2024-11-03  0.040337  False
18          1 2024-11-10  0.029779  False
19          1 2024-11-17  0.022271  False
20          1 2024-11-24  0.047095  False
21          1 2024-12-01  0.028343  False
22          1 2024-12-08  0.030189  False
23          1 2024-12-15  0.023487  False
24          1 2024-12-22  0.033322  False
25          1 2024-12-29  0.018133  False
26          1 2025-01-05  0.100539   True
27          2 2024-07-07  0.323733   True
28          2 2024-07-14  0.295456   True
29          2 2024-07-21  0.365500   True
30          2 2024-07-28  0.212622  False
31          2 2024-08-04  0.335854   True
32          2 2024-08-11  0.370852   True
33          2 2024-08-18  0.240400   True
34          2 2024-08-25  0.370652   True
35          2 2024-09-01  0.289700   True
36          2 2024-09-08  0.354036   True
37          2 2024-09-15  0.381004   True
38          2 2024-09-22  0.294248   True
39          2 2024-09-29  0.284618   True
40          2 2024-10-06  0.262627   True
41          2 2024-10-13  0.294848   True
42          2 2024-10-20  0.371352   True
43          2 2024-10-27  0.265762   True
44          2 2024-11-03  0.294248   True
45          2 2024-11-10  0.240400   True
46          2 2024-11-17  0.275614   True
47          2 2024-11-24  0.290500   True
48          2 2024-12-01  0.444445   True
49          2 2024-12-08  0.223733   True
50          2 2024-12-15  0.323233   True
51          2 2024-12-22  0.323733   True
52          2 2024-12-29  0.275614   True
53          2 2025-01-05  0.418971   True

### Interquartile Range (iqr)

In [37]:
%%time
tracker = CatDrift(df_reference, func="chebyshev", drift_limit='iqr', freq="W", time_col="datetime", id_col="unique_id", target_col="discrete_1", method="expanding")

CPU times: user 7.33 ms, sys: 0 ns, total: 7.33 ms
Wall time: 6.53 ms


In [38]:
%%time
tracker.predict(df_reference, time_col="datetime", id_col="unique_id", target_col="discrete_1")

CPU times: user 8.63 ms, sys: 0 ns, total: 8.63 ms
Wall time: 7.84 ms


unique_id   datetime    metric  drift
0           1 2024-01-07  0.023623  False
1           1 2024-01-14  0.032022  False
2           1 2024-01-21  0.027684  False
3           1 2024-01-28  0.043360  False
4           1 2024-02-04  0.032039  False
5           1 2024-02-11  0.026642  False
6           1 2024-02-18  0.042232  False
7           1 2024-02-25  0.031039  False
8           1 2024-03-03  0.018809  False
9           1 2024-03-10  0.036373  False
10          1 2024-03-17  0.039445  False
11          1 2024-03-24  0.054376  False
12          1 2024-03-31  0.034119  False
13          1 2024-04-07  0.022519  False
14          1 2024-04-14  0.032718  False
15          1 2024-04-21  0.044505  False
16          1 2024-04-28  0.069950   True
17          1 2024-05-05  0.014421  False
18          1 2024-05-12  0.028390  False
19          1 2024-05-19  0.045724  False
20          1 2024-05-26  0.022381  False
21          1 2024-06-02  0.037573  False
22          1 2024-06-09  0.036546  False
23          1 2024-06-16  0.025065  False
24          1 2024-06-23  0.053635  False
25          1 2024-06-30  0.026890  False
26          2 2024-01-07  0.297592   True
27          2 2024-01-14  0.370852   True
28          2 2024-01-21  0.379389   True
29          2 2024-01-28  0.259131   True
30          2 2024-02-04  0.251370   True
31          2 2024-02-11  0.311129   True
32          2 2024-02-18  0.251270   True
33          2 2024-02-25  0.402165   True
34          2 2024-03-03  0.240400   True
35          2 2024-03-10  0.263227   True
36          2 2024-03-17  0.302400   True
37          2 2024-03-24  0.311829   True
38          2 2024-03-31  0.276214   True
39          2 2024-04-07  0.390500   True
40          2 2024-04-14  0.250570   True
41          2 2024-04-21  0.275614   True
42          2 2024-04-28  0.378589   True
43          2 2024-05-05  0.305689   True
44          2 2024-05-12  0.248565   True
45          2 2024-05-19  0.266262   True
46          2 2024-05-26  0.229900   True
47          2 2024-06-02  0.282067   True
48          2 2024-06-09  0.289700   True
49          2 2024-06-16  0.252858   True
50          2 2024-06-23  0.286196   True
51          2 2024-06-30  0.338326   True

### Quantile Interval & Jacknife Method

In [39]:
%%time
tracker = CatDrift(df_reference, func="chebyshev", drift_limit=("quantile", None, 0.95), time_col="datetime", id_col="unique_id", target_col="discrete_1", freq="W", method="expanding")

CPU times: user 4.67 ms, sys: 3.79 ms, total: 8.46 ms
Wall time: 7.58 ms


# Continuous Data Drift

## Wasserstein Distance

In [40]:
%%time
ws = ConDrift(df_reference, func="ws", freq="W", drift_limit="auto", id_col="unique_id", target_col="feature_1", time_col="datetime")

CPU times: user 48.6 ms, sys: 3 μs, total: 48.6 ms
Wall time: 47.9 ms


In [41]:
%%time
ws.predict(df_reference, id_col="unique_id", target_col="feature_1", time_col="datetime")

CPU times: user 38.8 ms, sys: 3.99 ms, total: 42.8 ms
Wall time: 42 ms


unique_id   datetime    metric  drift
0           1 2024-01-07  0.045129  False
1           1 2024-01-14  0.059187  False
2           1 2024-01-21  0.099801  False
3           1 2024-01-28  0.042846  False
4           1 2024-02-04  0.076650  False
5           1 2024-02-11  0.030873  False
6           1 2024-02-18  0.070656  False
7           1 2024-02-25  0.053244  False
8           1 2024-03-03  0.046958  False
9           1 2024-03-10  0.055509  False
10          1 2024-03-17  0.057522  False
11          1 2024-03-24  0.036522  False
12          1 2024-03-31  0.037087  False
13          1 2024-04-07  0.070745  False
14          1 2024-04-14  0.100678  False
15          1 2024-04-21  0.055126  False
16          1 2024-04-28  0.120058   True
17          1 2024-05-05  0.077501  False
18          1 2024-05-12  0.050361  False
19          1 2024-05-19  0.095305  False
20          1 2024-05-26  0.047498  False
21          1 2024-06-02  0.047539  False
22          1 2024-06-09  0.056493  False
23          1 2024-06-16  0.056129  False
24          1 2024-06-23  0.120019   True
25          1 2024-06-30  0.050722  False
26          2 2024-01-07  0.222888  False
27          2 2024-01-14  0.166462  False
28          2 2024-01-21  0.382617   True
29          2 2024-01-28  0.481356   True
30          2 2024-02-04  0.174165  False
31          2 2024-02-11  0.223190  False
32          2 2024-02-18  0.213583  False
33          2 2024-02-25  0.259306  False
34          2 2024-03-03  0.282569  False
35          2 2024-03-10  0.163918  False
36          2 2024-03-17  0.285494  False
37          2 2024-03-24  0.170713  False
38          2 2024-03-31  0.317947  False
39          2 2024-04-07  0.379450   True
40          2 2024-04-14  0.141373  False
41          2 2024-04-21  0.283099  False
42          2 2024-04-28  0.263223  False
43          2 2024-05-05  0.187391  False
44          2 2024-05-12  0.220429  False
45          2 2024-05-19  0.160015  False
46          2 2024-05-26  0.239718  False
47          2 2024-06-02  0.136629  False
48          2 2024-06-09  0.205847  False
49          2 2024-06-16  0.279296  False
50          2 2024-06-23  0.254374  False
51          2 2024-06-30  0.169946  False

In [42]:
%%time
ws = ConDrift(df_reference, func="ws", freq="W", drift_limit="auto", id_col="unique_id", target_col="feature_1", time_col="datetime")

CPU times: user 54 ms, sys: 23 μs, total: 54 ms
Wall time: 52.9 ms


In [43]:
%%time
ws.predict(df_test, id_col="unique_id", target_col="feature_1", time_col="datetime")

CPU times: user 39.8 ms, sys: 3 μs, total: 39.8 ms
Wall time: 38.9 ms


unique_id   datetime    metric  drift
0           1 2024-07-07  0.050105  False
1           1 2024-07-14  0.079363  False
2           1 2024-07-21  0.141737   True
3           1 2024-07-28  0.062830  False
4           1 2024-08-04  0.027785  False
5           1 2024-08-11  0.068626  False
6           1 2024-08-18  0.038329  False
7           1 2024-08-25  0.047665  False
8           1 2024-09-01  0.111672   True
9           1 2024-09-08  0.039121  False
10          1 2024-09-15  0.062662  False
11          1 2024-09-22  0.054522  False
12          1 2024-09-29  0.049211  False
13          1 2024-10-06  0.071946  False
14          1 2024-10-13  0.040054  False
15          1 2024-10-20  0.047172  False
16          1 2024-10-27  0.058202  False
17          1 2024-11-03  0.109494   True
18          1 2024-11-10  0.038385  False
19          1 2024-11-17  0.060108  False
20          1 2024-11-24  0.036321  False
21          1 2024-12-01  0.036885  False
22          1 2024-12-08  0.080883  False
23          1 2024-12-15  0.058380  False
24          1 2024-12-22  0.047122  False
25          1 2024-12-29  0.104227  False
26          1 2025-01-05  0.095861  False
27          2 2024-07-07  0.435131   True
28          2 2024-07-14  0.273622  False
29          2 2024-07-21  0.290029  False
30          2 2024-07-28  0.248699  False
31          2 2024-08-04  0.182156  False
32          2 2024-08-11  0.350838  False
33          2 2024-08-18  0.240485  False
34          2 2024-08-25  0.219609  False
35          2 2024-09-01  0.158217  False
36          2 2024-09-08  0.191439  False
37          2 2024-09-15  0.242217  False
38          2 2024-09-22  0.271660  False
39          2 2024-09-29  0.416663   True
40          2 2024-10-06  0.153967  False
41          2 2024-10-13  0.375245   True
42          2 2024-10-20  0.455304   True
43          2 2024-10-27  0.175006  False
44          2 2024-11-03  0.243505  False
45          2 2024-11-10  0.237213  False
46          2 2024-11-17  0.251346  False
47          2 2024-11-24  0.328630  False
48          2 2024-12-01  0.206726  False
49          2 2024-12-08  0.222759  False
50          2 2024-12-15  0.305077  False
51          2 2024-12-22  0.277341  False
52          2 2024-12-29  0.170350  False
53          2 2025-01-05  0.343773  False